## 梯度下降法进行逻辑回归
### 生成逻辑回归的样本数据
逻辑回归的函数形式：
$Y = f(z)=\frac{1}{1+e^{z}}, z=\beta X+\varepsilon $

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl

In [2]:
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score

In [3]:
data = make_classification(n_samples=3200,n_features=4,n_informative=4,n_redundant=0)

In [4]:
x = data[0].T
Y = data[1]

In [5]:
def Beta(X,b):#X为X向量，b为β向量
    X = np.vstack((1,X.reshape(len(X),1)))
    Y = np.dot(b.reshape(1,len(X)),X)
    return Y[0][0]

In [6]:
#自造逻辑回归样本，效果并不好，考虑是因为X的值过于远离0点，Y值的变化概率很小，导致损失函数的减小的向量过小，无法迭代优化。
'''def random_log(beta_range,x_num,n,x_range,seed,var_epsilon = 1):#生成线性回归数据
    np.random.seed(seed)
    beta = np.random.uniform(beta_range[0],beta_range[1],x_num+1)
    np.random.seed(seed)
    x = np.random.uniform(x_range[0],x_range[1],x_num*n)
    x = x.reshape(x_num, n)
    z = []
    for i in range(n):
        X = np.array([x[0:,i]]).reshape(x_num, 1)
        z.append(Beta(X, beta))
    np.random.seed(seed)
    epsilon = np.random.normal(0, var_epsilon, n)
    z = z + epsilon
    Y = np.around(1/(1+np.exp(z)))
    return {"beta":beta, "x":x, "Y":Y}'''

'def random_log(beta_range,x_num,n,x_range,seed,var_epsilon = 1):#生成线性回归数据\n    np.random.seed(seed)\n    beta = np.random.uniform(beta_range[0],beta_range[1],x_num+1)\n    np.random.seed(seed)\n    x = np.random.uniform(x_range[0],x_range[1],x_num*n)\n    x = x.reshape(x_num, n)\n    z = []\n    for i in range(n):\n        X = np.array([x[0:,i]]).reshape(x_num, 1)\n        z.append(Beta(X, beta))\n    np.random.seed(seed)\n    epsilon = np.random.normal(0, var_epsilon, n)\n    z = z + epsilon\n    Y = np.around(1/(1+np.exp(z)))\n    return {"beta":beta, "x":x, "Y":Y}'

In [7]:
'''x = random_log([-5,5], 4, 3200, [-100,100], 431)['x']
beta = random_log([-5,5], 4, 3200, [-100,100], 431)['beta']
Y = random_log([-5,5], 4, 3200, [-100,100], 431)['Y']'''

"x = random_log([-5,5], 4, 3200, [-100,100], 431)['x']\nbeta = random_log([-5,5], 4, 3200, [-100,100], 431)['beta']\nY = random_log([-5,5], 4, 3200, [-100,100], 431)['Y']"

### 确定损失函数$J(\hat{\beta})$
损失函数设为对数极大似然函数的相反数。

极大似然函数：
$L(\hat{\beta} )=\prod_{i=1}^{n}p_{0}^{1-y_{i} }p_{1}^{y_{i} }$,
    其中p0，p1分别表示$\hat{y_{i}}$取0和1的概率。

$p_{1}=\frac{1}{1+e^{\hat{\beta}X_{i} }} ,p_{0}=1-p_{1}=\frac{e^{\hat{\beta}X_{i}}}{1+e^{\hat{\beta}X_{i} }}$

对数似然函数：$lnL(\hat{\beta})=\sum_{i=1}^{n}ln(p_{0}^{1-y_{i}})+\sum_{i=1}^{n}ln(p_{1}^{y_{i}})$

$=\sum_{i=1}^{n}(1-y_{i})ln(e^{\hat{\beta}X_{i}})-\sum_{i=1}^{n}(1-y_{i})ln(1+e^{\hat{\beta}X_{i}})-\sum_{i=1}^{n}y_{i}ln(1+e^{\hat{\beta}X_{i}}) $

$=\sum_{i=1}^{n}(1-y_{i})\hat{\beta}X_{i}-\sum_{i=1}^{n}ln(1+e^{\hat{\beta }X_{i} })$

所以$J(\hat{\beta})=-lnL(\hat{\beta})=\sum_{i=1}^{n}ln(1+e^{\hat{\beta }X_{i} })-\sum_{i=1}^{n}(1-y_{i})\hat{\beta}X_{i}$

### 对损失函数求$\hat{\beta}$的偏导数

$J'(\hat{\beta})=\left\{\begin{matrix} 
  \sum_{i=1}^{n} (\frac{e^{\hat{\beta}X_{i}}}{1+e^{\hat{\beta}X_{i}}} -  1+y_{i}) (对\beta_{0}求导)\\  
   \sum_{i=1}^{n} [X_{ij}(\frac{e^{\hat{\beta}X_{i}}}{1+e^{\hat{\beta}X_{i}}} -  1+y_{i})(对\beta_{j}求导)
\end{matrix}\right. $

In [8]:
def partical(x, beta_hat, Y):#βhat为估计的β向量
    sum0 = []
    k = len(x[0:,0])#变量数
    n = len(x[0, 0:])#样本数
    for i in range(n):
        sum0.append(np.exp(Beta(x[0:,i],beta_hat))/(1+np.exp(Beta(x[0:,i],beta_hat))) - 1 + Y[i])
    vector0 = sum(sum0)
    vector = []
    for j in range(k):
        vector.append(sum(np.asarray(sum0) * x[j,0:]))
    vector = np.hstack((vector0,vector))
    return vector

### 小批量梯度下降法迭代估计$\beta$的值，并计算auc查看模型拟合效果

In [9]:
def mngb(x, Y, mini_batch, origin, accuracy, alpha,times_num = 1000000):
    col = np.random.randint(len(x[0, 0:]), size = mini_batch)
    x0 = x[0: , col]
    Y0 = Y[col]
    theta = origin - alpha * partical(x0, origin, Y0)
    delta = accuracy +1
    times = 0
    while delta >= accuracy:
        times += 1
        col = np.random.randint(len(x[0, 0:]), size = mini_batch)
        x0 = x[0: , col]
        Y0 = Y[col]
        theta = theta - alpha * partical(x0, theta, Y0)
        delta = alpha * abs(max(partical(x0, theta, Y0)))
        if times >= times_num: break
    return theta

In [20]:
beta_hat = mngb(x, Y, 4, np.array([0, 0, 0, 0, 0]), 1e-8, 1e-3, 100000)
print(beta_hat)

[-0.85976649  0.92891547  0.73281237 -1.01556071  2.39539702]


In [11]:
def yhat(x,beta):
    z = []
    for i in range(len(x[0,0:])):
        z.append(Beta(x[0:,i],beta))
    y = np.around(1/(1+np.exp(z)))
    return y

In [21]:
y_hat = yhat(x,beta_hat)
roc_auc_score(Y, y_hat)

0.9378036866420651

In [13]:
from sklearn.linear_model import LogisticRegression

In [15]:
lr = LogisticRegression()

In [17]:
re = lr.fit(x.T,Y)

In [18]:
r = re.score(x.T,Y)

In [19]:
r

0.9453125